In [ ]:
import numpy as np
import cv2
import copy
import asynchio

def findWhitePixels(diff, threshold, height, width):
    side1 = 30
    side2 = 40
    a = np.array([])
    for i in range(0, height - side1, side1):
        for j in range(0, width - side2, side2):
            diff_m = (diff[i:(i + side1), j:(j + side2)]) >= 15
            if np.sum(diff_m) > threshold:
                a = np.append(a, [i, j, i + side1, j + side2])
    return a
            

cap = cv2.VideoCapture(1)


frames_counter = 0 #stores the number of frames displayed since launch
font = cv2.FONT_HERSHEY_SIMPLEX
interval = 5 #delay between two frames that are substracted to produce the final image
samples_number = 15
delay = 30 
samples = np.array([])
calibration = -1
highest = 0
square_positions = [None] * delay
frames = [None] * interval
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output15.avi', fourcc, 10.0, (640,480), 0)

while True:
    ret, frame = cap.read()
    height, width, t = frame.shape
    if ret==True:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_np = np.array(frame)
        backup = np.zeros(frame.shape)
            
        if frames_counter < interval:
            frames[frames_counter] = frame_np
        else:
            backup = frames[frames_counter % interval]
            frames[frames_counter % interval] = frame_np

        frame_np = frame_np.astype('int32')
        backup = backup.astype('int32')
        diff = np.abs(frame_np - backup) 
        diff = diff.astype('uint8')
        frame_np = frame_np.astype('uint8')
        backup = backup.astype('uint8')

        filled_squares = findWhitePixels(diff, 8, 480, 640)
        filled_squares = filled_squares.astype('int32')
        for i in range(0, filled_squares.shape[0], 4):
            if (filled_squares[i] < filled_squares[highest]) and (filled_squares.shape[0] != 0):
                highest = i
        if filled_squares.shape[0] != 0:
            diff[filled_squares[highest]:filled_squares[highest + 2], filled_squares[highest + 1]:filled_squares[highest + 3]] = 255
        
            if (frames_counter >= interval) and ((frames_counter - interval) < samples_number):
                samples = np.append(samples, filled_squares[highest])
                
            if ((frames_counter - interval) == samples_number):
                calibration = np.average(samples)
            #print (samples.shape[0])
            #print (calibration)
            if calibration != -1 and filled_squares.shape[0] != 0:
                print(filled_squares[highest] - calibration)
                print (3)
                cv2.putText(diff, str(filled_squares[highest] - calibration), (10, 100), font, 3, (255, 255, 0), 2, cv2.LINE_AA)
        
        out.write(diff)
        
            
        cv2.imshow('frame',diff)
        frames_counter += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()

out.release()

cv2.destroyAllWindows()

In [ ]:
import pyglet

music = pyglet.resource.media('music.mp3', streaming = False)
player = pyglet.media.Player()
player.queue(music)
#music.play()
player.play()
print (3)
player.pause()
pyglet.app.run()

In [4]:
import pygame
print ('imported')
pygame.init()
print ('init')
pygame.mixer.music.load("music.wav")
print ('loaded')

pygame.mixer.music.set_volume(0.5)
pygame.mixer.music.play()
print ('playing')

clock = pygame.time.Clock()
clock.tick(10)
pygame.mixer.music.stop()
print ('stopped')

imported
init
loaded
playing
stopped
